In [1]:
import numpy as np
from numpy.linalg import inv

predict step: 
returns new state vector X and the new P variance

In [2]:
def predict(F, X, B, u, P, Q, verbos=0):
    # Project the state ahead
    X_new = F*X + B*u
    # Project the error covariance ahead
    P_new = F*P*F.T + Q
    
    if verbos > 1:
        print("X_new = {}*{} + {}*{}".format(F,X,B,u))
        print("P_new = {}*{}*{} + {}".format(F,P,F.T,Q))
        
    if verbos > 0:
        print("X': ", X_new)
        print("P': ", P_new)
    
    return X_new, P_new

gets a new measurement from the system and refines the guess

In [3]:
def measure(P_new, H, R, X_new, z, verbos=0):
    # Compute the Kalman gain
    K = P_new * H.T * inv(H * P_new * H.T + R)
    # Update estimate with measurement z
    X_new = X_new + K * (z - H * X_new)
    # Update the error covariance
    I = np.eye(P_new.shape[0], P_new.shape[1])
    P_new = (I - K * H) * P_new
    
    if verbos > 1:
        print("K = {}*{}*inv({}*{}*{} + {})".format(P_new,H.T,H,P_new,H.T,R))
        print("X_new = {} + {}*({} - {} * {})".format(X_new,K,z,H,X_new))
        print("P_new = (I - {}*{})*{}".format(K,H,P_new))
        
#     if verbos > 0:
#         print('\nAfter measurement:\n', z)
#         print('_______________________')
#         print('\nX_new:\n', X_new)
#         print('_______________________')
#         print('\nP_new:\n', P_new)
#         print('_______________________')
#         print('\nKalman gain:\n', K)
        
    return X_new, P_new, K


# Q3:

In [4]:
class KalmanFilter:
    def __init__(self, F, B, u, Q, H, R, X, P):
        self.F = F  # state transistion matrix
        self.B = B  # control transition matrix
        self.u = u
        self.Q = Q  # Process uncertainty/noise
        self.H = H  # measurement function
        self.R = R  # measurement uncertainty/noise
        self.X_current = X  # filter state estimate
        self.P_current = P  # covariance matrix
        self.K_gain = None  # Kalman gain

    def run(self, z, verbos=0):
        # z = vector of measurements
        z_counter = 1
        for m in z:
            if verbos > 0:
                print(z_counter, ':')
                z_counter = z_counter + 1
            self.X_current, self.P_current = predict(self.F, self.X_current, self.B, self.u, self.P_current, self.Q, verbos)
            self.X_current, self.P_current, self.K_gain = measure(self.P_current, self.H, self.R, self.X_current, m, verbos)
                
        print('Finish!')

        print('\nState Vector is:\n', self.X_current)
        print('_______________________')
        print('\nP matrix is:\n', self.P_current)
        print('_______________________')
        print('\nKalman gain is:\n', self.K_gain)

# Q1:

In [5]:
P = np.matrix([[2**2, 0        ]
              ,[0,         1.2**2]])

deltaT = 1.0
F = np.matrix([[1, deltaT]
              ,[0, 1     ]])

X = np.matrix([[8],
               [5]])

print('After Prediction:')
Q = 0
B = 0
u = 0
X, P = predict(F, X, B, u, P, Q)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print('_________________')

print('After Update:')
R = np.matrix([ 0.5**2])
H = np.matrix([1./0.3048 , 0])
z = np.matrix([43])
X, P, K_gain = measure(P, H, R, X, z)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print("\nKalman Gain is:\n", K_gain)

After Prediction:

P matrix is:
 [[5.44 1.44]
 [1.44 1.44]]

State Vector is:
 [[13.]
 [ 5.]]
_________________
After Update:

P matrix is:
 [[0.02312702 0.00612186]
 [0.00612186 1.06044402]]

State Vector is:
 [[13.10594766]
 [ 5.02804497]]

Kalman Gain is:
 [[0.30350421]
 [0.08033935]]


# Q2:

In [6]:
P = np.matrix([[2**2, 0     ]
              ,[0   , 1.2**2]])

delta_t = 1.0
F = np.matrix([[1, deltaT]
              ,[0, 1     ]])

X = np.matrix([[8],
               [5]])

print('After Prediction:')
Q = 0
B = 0
u = 0
X, P = predict(F, X, B, u, P, Q)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print('_________________')

print('After Update:')
R = np.matrix([[0.5**2, 0   ],
               [0     , 4**2]])
H = np.matrix([[1./0.3048 , 0],
               [0         , 1.0]])
z = np.matrix([[43]
              ,[4]])
X, P, K_gain = measure(P, H, R, X, z)
print("\nP matrix is:\n", P)
print("\nState Vector is:\n", X)
print("\nKalman Gain is:\n", K_gain)

After Prediction:

P matrix is:
 [[5.44 1.44]
 [1.44 1.44]]

State Vector is:
 [[13.]
 [ 5.]]
_________________
After Update:

P matrix is:
 [[0.02312482 0.00574134]
 [0.00574134 0.99452888]]

State Vector is:
 [[13.10557877]
 [ 4.96414369]]

Kalman Gain is:
 [[0.30347538 0.00035883]
 [0.07534561 0.06215806]]


# Q4:

In [7]:
delta_t = 1.0

P = np.matrix([ [7**2, 0   , 0     , 0],
                [0   , 7**2, 0     , 0],
                [0   , 0   , 100**2, 0],
                [0   , 0   , 0     , 100**2]])

F = np.matrix([ [1, 0, delta_t, 0      ],
                [0, 1, 0      , delta_t],
                [0, 0, 1      , 0      ],
                [0, 0, 0      , 1      ]])

X = np.matrix([ [200],
                [150],
                [0],
                [0]])

H = np.matrix([[1, 0, 0, 0],
               [0, 1, 0, 0]])

R = np.matrix([ [6**2,0   ],
                [0   ,6**2]])

u = np.matrix([ [0],
                [0],
                [5],
                [15]])

B = np.matrix([[0,0,0.5,0  ],
               [0,0,0  ,0.5],
               [0,0,1  ,0  ],
               [0,0,0  ,1  ]])

Q = np.zeros_like(P)

# sensor measurements
z = [np.matrix([[240], [204]]),
     np.matrix([[284], [267]]),
     np.matrix([[334], [344]]),
     np.matrix([[390], [437]]),
     np.matrix([[450], [544]]),
     np.matrix([[516], [667]])]

kalman_filter = KalmanFilter(F, B, u, Q, H, R, X, P)
kalman_filter.run(z)
# X_roi = [515.30128021 667.82815584  67.6560699  130.62964398]

Finish!

State Vector is:
 [[515.30128021]
 [667.82815584]
 [ 67.6560699 ]
 [130.62964398]]
_______________________

P matrix is:
 [[17.059979    0.          4.06497694  0.        ]
 [ 0.         17.059979    0.          4.06497694]
 [ 4.06497694  0.          1.41055982  0.        ]
 [ 0.          4.06497694  0.          1.41055982]]
_______________________

Kalman gain is:
 [[0.47388831 0.        ]
 [0.         0.47388831]
 [0.11291603 0.        ]
 [0.         0.11291603]]


# 2020 moed b:

In [8]:
delta_t = 0.1

P = np.matrix([ [18**2, 0],
                [0   , 8**2]])

F = np.matrix([ [1, delta_t],
                [0, 1]])

X = np.matrix([ [300],
                [25]])

H = np.matrix([[1/2, 0],
               [0, 3]])

R = np.matrix([ [10**2,0   ],
                [0   ,8**2]])

u = np.matrix([ [10]])

B = np.matrix([[ 0.5* delta_t**2],
               [ delta_t ]])

Q = np.zeros_like(P)

# sensor measurements
z = [np.matrix([[170],
                [80]])]

kalman_filter = KalmanFilter(F, B, u, Q, H, R, X, P)
kalman_filter.run(z, verbos=1)

1 :
X':  [[302.55]
 [ 26.  ]]
P':  [[324.64   6.4 ]
 [  6.4   64.  ]]
Finish!

State Vector is:
 [[319.34436735]
 [ 26.63304901]]
_______________________

P matrix is:
 [[179.02505856   0.35355991]
 [  0.35355991   6.3994343 ]]
_______________________

Kalman gain is:
 [[0.89512529 0.01657312]
 [0.0017678  0.29997348]]


# 2021 moed a:

In [9]:
delta_t = 2

P = np.matrix([ [5**2, 0],
                [0   , 1.2**2]])

F = np.matrix([ [1, delta_t],
                [0, 1]])

X = np.matrix([ [30],
                [2.5]])

H = np.matrix([[100, 0]])

R = np.matrix([[25**2]])

u = np.matrix([[3]])

B = np.matrix([[ 0.5* delta_t**2],
               [ delta_t ]])

Q = np.matrix([ [2, 0],
                [0, 1]])

# sensor measurements
z = [np.matrix([[370]])]

kalman_filter = KalmanFilter(F, B, u, Q, H, R, X, P)
kalman_filter.run(z, verbos=0)

Finish!

State Vector is:
 [[3.77102597]
 [5.22712316]]
_______________________

P matrix is:
 [[0.06238099 0.00548404]
 [0.00548404 2.1872953 ]]
_______________________

Kalman gain is:
 [[0.00998096]
 [0.00087745]]


# 2021 moed b:

In [10]:
delta_t = 2

P = np.matrix([ [5**2, 0],
                [0   , 1.2**2]])

F = np.matrix([ [1, delta_t],
                [0, 1]])

X = np.matrix([ [300],
                [12.5]])

H = np.matrix([[1/1000, 0],
               [0, 1]])

R = np.matrix([ [0.01**2,0   ],
                [0   ,3**2]])

u = np.matrix([ [-5]])

B = np.matrix([[ 0.5* delta_t**2],
               [ delta_t ]])

Q = np.matrix([ [2, 0],
                [0, 1]])

# sensor measurements
z = [np.matrix([[0.321],
                [2]])]

kalman_filter = KalmanFilter(F, B, u, Q, H, R, X, P)
kalman_filter.run(z, verbos=1)

1 :
X':  [[315. ]
 [  2.5]]
P':  [[32.76  2.88]
 [ 2.88  2.44]]
Finish!

State Vector is:
 [[316.36041523]
 [  2.49847731]]
_______________________

P matrix is:
 [[24.2624861   1.7160108 ]
 [ 1.7160108   1.88070017]]
_______________________

Kalman gain is:
 [[2.42624861e+02 1.90667867e-01]
 [1.71601080e+01 2.08966686e-01]]
